In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117


In [ ]:
!pip install transformers[sentencepiece] datasets spacy scipy networkx numpy sent2vec pyngrok pandas nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=bab48b98ecdb122f45554d018ce0c822812f4ab079d1b9895f69c02e631f2b1f
  Sto

In [ ]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ff24d01b49a991cff48eb2303ef742402326d6f788457e8e599720dd5a688e22
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# import library yang dibutuhkan
import spacy
import networkx as nx
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import numpy as np

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import TFPegasusForConditionalGeneration, PegasusTokenizerFast

import re
import unicodedata

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**PERINGKASAN BAHASA INGGRIS**

In [ ]:
# 1. FUNGSI PRE-PROCESSING
def preprocess_en(text):
  nlp = spacy.load("en_core_web_sm")
  doc = nlp(text)

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = [sent.text.strip() for sent in doc.sents]

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):
    doc = nlp(sentences[i])

    # inisial variabel untuk menampung kata yang bersih
    filtered_tokens = []

    # tokenisasi
    for token in doc:
    # menghilangkan stopword dan karakter yang tidak dibutuhkan
      if token.is_stop or token.is_punct:
        continue
      # lemmatisasi
      filtered_tokens.append(token.lemma_)

    # how to convert list to string
    result = " ".join(filtered_tokens)

    filtered_sentences.append(result)
  return filtered_sentences

In [ ]:
# Load the tokenizer and model
tokenizer_bert_en = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert_en = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
def vector_en(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [ ]:
# 2. FUNGSI PERINGKASAN EKSTRAKTIF
def extractive_sum_en(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_en(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]
  summary = " ".join(summary)

  # hasil peringkasan ekstraktif
  return summary


In [ ]:
# 3. FUNGSI PERINGKASAN ABSTRAKTIF

model_name1 = "google/pegasus-cnn_dailymail"
model1 = PegasusForConditionalGeneration.from_pretrained(model_name1)
tokenizer1 = PegasusTokenizer.from_pretrained(model_name1)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def abstractive_sum_en(text):

  inputs = tokenizer1.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)

  summary_ids = model1.generate(inputs, max_length=350, min_length=50, length_penalty=2.0, num_beams=6, early_stopping=True)

  summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)

  return summary

In [ ]:
# MERINGKAS TEKS BAHASA INGGRIS
def summy_en(text, n):
  document = text
  # load spacy
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary1 = extractive_sum_en(result, sentences, n)

  # melakukan peringkasan abstraktif
  summary = abstractive_sum_en(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary


In [ ]:
text = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [ ]:
result_en = summy_en(text, 2)

In [ ]:
print(result_en)

Python supports object-oriented programming, functional programming, and procedural programming paradigms.
Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation.
 Python is an essential skill for many industries and fields.


**PERINGKASAN BAHASA INDONESIA**

In [ ]:
# Load the tokenizer and model
tokenizer_bert_id = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_bert_id = BertModel.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def vector_id(list_sentences):

  # Initialize an empty array to store sentence embeddings
  sentence_embeddings = []

  # Iterate through sentences and get embeddings
  for text in list_sentences:
      input_ids = tokenizer_bert_en.encode(text, add_special_tokens=True, padding=True, truncation=True)
      input_ids = torch.tensor(input_ids).unsqueeze(0)

      with torch.no_grad():
          outputs = model_bert_en(input_ids)
          cls_embedding = outputs.last_hidden_state[:, 0, :]
          embedding = cls_embedding[0]

      sentence_embeddings.append(embedding.numpy())

  return sentence_embeddings

In [ ]:
#1. PRE-PROCESSING
def preprocess_id(text):

  # Segmentasi (pisahkan text per kalimat dan masukkan kedalam list)
  sentences = sent_tokenize(text)

  # Membersihkan setiap kalimat
  filtered_sentences = []
  for i in range(len(sentences)):

    text = sentences[i]

    # Tokenisasi (memecah kalimat menjadi kata)
    words = word_tokenize(text.lower())

    # Membersihkan tanda baca
    words = [word for word in words if word.isalnum()]

    # Menghapus stopwords
    stop_words = set(stopwords.words("indonesian"))
    words = [word for word in words if word not in stop_words]

    # Melakukan lemmatisasi
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Menggabungkan kembali kata yang telah dibersihkan menjadi kalimat
    result = " ".join(lemmatized_words)

    filtered_sentences.append(result)
  return filtered_sentences

In [ ]:
# 2. PERINGKASAN EKSTRAKTIF

def extractive_sum_id(filtered_sentences, sentences, n):
  # mengubah kalimat menjadi vektor
  vectors = vector_id(filtered_sentences)

  # menghitung similarity matrix (matriks kemiripan antar kalimat)
  similarity_matrix = []
  for i in range(len(vectors)):
    row = []
    for j in range(len(vectors)):
      row.append(spatial.distance.cosine(vectors[i], vectors[j]))
    similarity_matrix.append(row)

  # konversi matrix menjadi graph
  graph = nx.from_numpy_array(np.array(similarity_matrix))

  # melakukan perangkingan
  scores = nx.pagerank(graph)

  # mengambil top rank (kalimat dengan score tertinggi)
  sentences_size = len(sentences)
  num_sentences = round((sentences_size + 1) / n)

  top_sentence_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:num_sentences]
  summary = [sentences[i] for i in top_sentence_indices]

  # hasil peringkasan ekstraktif

  return " ".join(summary)

In [ ]:
# 3. PERINGKASAN ABSTRAKTIF

# membersihkan teks
def text_cleaning(input_string):
    lowercase = input_string.lower()
    remove_link = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)', '', lowercase).replace("&amp;","&")
    remove_bullet = "\n".join([T for T in remove_link.split('\n') if '•' not in T and "baca juga:" not in T])
    remove_accented = unicodedata.normalize('NFKD', remove_bullet).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    remove_parentheses = re.sub("([\(\|]).*?([\)\|])", "\g<1>\g<2>", remove_accented)
    remove_punc = re.sub(r"[^\w\d.\s]+",' ', remove_parentheses)
    remove_num_dot = re.sub(r"(?<=\d)\.|\.(?=\d)|(?<=#)\.","", remove_punc)
    remove_extra_whitespace =  re.sub(r'^\s*|\s\s*', ' ', remove_num_dot).strip()
    return ".".join([s for s in remove_extra_whitespace.strip().split('.') if len(s.strip())>10]).replace("_","")

In [ ]:
model_name2 = "thonyyy/pegasus_indonesian_base-finetune"
model2 = TFPegasusForConditionalGeneration.from_pretrained(model_name2)
tokenizer2 = PegasusTokenizerFast.from_pretrained(model_name2)

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at thonyyy/pegasus_indonesian_base-finetune.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def abstractive_sum_id(text):
  clean_text = text_cleaning(text)
  inputs = tokenizer2(clean_text, return_tensors = 'tf')
  summary_ids = model2.generate(**inputs, max_new_tokens = 256, length_penalty=2.0, num_beams=4, early_stopping=True)
  summary = tokenizer2.batch_decode(summary_ids, skip_special_tokens=True)

  return summary[0]

In [ ]:
# MERINGKAS TEKS BAHASA
def summy_id(text, n):

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  sentences = sent_tokenize(text)

  # 1. PRE-PROCESSING
  result = preprocess_id(text)

  # 2. PERINGKASAN EKSTRAKTIF
  summary1 = extractive_sum_id(result, sentences, n)

  # 3. PERINGKASAN ABSTRAKTIF
  summary = abstractive_sum_id(summary1)
  summary = summary.replace("<pad>", "")
  summary = summary.replace("</s>", "")
  summary = summary.replace(".<n>", ".\n")

  return summary

In [ ]:
text = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [ ]:
result_id = summy_id(text, 2)

Initializing Bert distilbert-base-multilingual-cased
Vectorization done on cpu


In [ ]:
print(result_id)

sampah menjadi salah satu masalah kesehatan yang paling banyak ditemui di indonesia. banyak orang menjadi sadar bahwa pengelolaan sampah sangatlah penting untuk menjamin rasa nyaman lingkungan serta memperhatikan kesehatan


**PERINGKASAN BAHASA INGGRIS PENGUJIAN**

In [ ]:
# 1. TEXTRANK
def f_textrank_en(text, n):
  document = text
  nlp = spacy.load("en_core_web_sm")

  # memasukkan kalimat kedalam list (untuk memudahkan print output)
  doc = nlp(document)
  sentences = [sent.text.strip() for sent in doc.sents]

  # melakukan pre-processing
  # contoh hasil penggunaan fungsi preprocess
  result = preprocess_en(document)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_en(result, sentences, n)

  return summary

In [ ]:
# 2. PEGASUS
def f_pegasus_en(text):
  summary = abstractive_sum_en(text)
  return summary

In [ ]:
# 3. HYBRID
def f_hybrid_en(text, n):
  summary = summy_en(text, n)
  return summary

In [ ]:
text1 = """

Python is a high-level, interpreted programming language that is widely used for web development, data analysis, artificial intelligence, and many other applications. Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages in the world, thanks to its simplicity, versatility, and wide range of libraries and frameworks.

Python is known for its elegant syntax, which emphasizes readability and reduces the cost of program maintenance. Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. This makes Python code more concise and easier to read, especially for beginners who are just learning how to program.

In addition to its simplicity and readability, Python is also praised for its powerful and flexible features. Python supports object-oriented programming, functional programming, and procedural programming paradigms, making it a versatile tool for different kinds of applications. Python also has a large standard library and a vast ecosystem of third-party libraries and packages, which makes it easy to find and use pre-built modules for different purposes.

Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields. So why not start learning Python today and see where it takes you?

"""

In [ ]:
hasil_textrank_en = f_textrank_en(text1, 3)
hasil_pegasus_en = f_pegasus_en(text1)
hasil_hybrid_en = f_hybrid_en(text1, 2)

In [ ]:
hasil_textrank_en

'Unlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation. So why not start learning Python today and see where it takes you? Whether you are a beginner or an experienced programmer, learning Python can open up many opportunities for you. With its growing popularity and wide range of applications, Python is an essential skill for many industries and fields.'

In [ ]:
hasil_pegasus_en

'Python is a high-level, interpreted programming language.<n>It is widely used for web development, data analysis, artificial intelligence, and many other applications.<n>Created by Guido van Rossum in the late 1980s, Python has become one of the most popular programming languages.'

In [ ]:
hasil_hybrid_en

'Python supports object-oriented programming, functional programming, and procedural programming paradigms.\nUnlike other programming languages that use curly braces and semicolons to denote code blocks and statements, Python uses whitespace and indentation.\n Python is an essential skill for many industries and fields.'

**PERINGKASAN BAHASA INDONESIA PENGUJIAN**

In [ ]:
# 1. TEXTRANK
def f_textrank_id(text, n):
  sentences = sent_tokenize(text)

  result = preprocess_id(text)

  # melakukan peringkasan ekstraktif
  summary = extractive_sum_id(result, sentences, n)

  return summary

In [ ]:
# 2. PEGASUS
def f_pegasus_id(text):
  summary = abstractive_sum_id(text)
  return summary

In [ ]:
# 3. HYBRID
def f_hybrid_id(text, n):
  summary = summy_id(text, n)
  return summary

In [ ]:
text2 = """

Sampah selalu kita temukan mengotori lingkungan di sekitar kita. Maka wajar karena hal itu seringkali sampah menjadi masalah lingkungan yang serius harus ditangani.

Sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. Walaupun sampah jelas-jelas membuat lingkungan tidak nyaman tetapi anehnya kesadaran kita terhadap lingkungan masih jauh dari cukup.

Masih banyak di antara kita yang tidak memperhatikan membuang sampah pada tempatnya. Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk.

Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan.

"""

In [ ]:
hasil_textrank_id = f_textrank_id(text2, 3)
hasil_pegasus_id = f_pegasus_id(text2)
hasil_hybrid_id = f_hybrid_id(text2, 2)

In [ ]:
hasil_textrank_id

'Mereka baru menyadari pentingnya membuang sampah secara disiplin, ketika mulai banyak rusaknya lingkungan diakibatkan oleh sampah yang menumpuk. Pada akhirnya kondisi ini telah membuat banyak orang menjadi sadar bahwa mengelola sampah dengan bijak sangatlah penting untuk menjamin rasa nyaman lingkungan juga memperhatikan kesehatan. \n\nSampah selalu kita temukan mengotori lingkungan di sekitar kita.'

In [ ]:
hasil_pegasus_id

'sampah bisa membuat suasana nyaman menjadi rusak seketika karena bau sampah yang menyengat. padahal sampah jelas jelas membuat lingkungan tidak nyaman tetapi anehnya membuat lingkungan masih jauh dari cukup banyak'

In [ ]:
hasil_hybrid_id

'sampah menjadi salah satu masalah kesehatan yang paling banyak ditemui di indonesia. banyak orang menjadi sadar bahwa pengelolaan sampah sangatlah penting untuk menjamin rasa nyaman lingkungan serta memperhatikan kesehatan'

**PENGUJIAN PADA MASING MASING DATASET**

PENGUJIAN PADA 50 ARTIKEL CNN/DAILY MAIL

In [ ]:
import evaluate

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_cnn = load_dataset("cnn_dailymail", version="3.0.0")

print(f"Features in cnn_dailymail : {dataset_cnn['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features in cnn_dailymail : ['article', 'highlights', 'id']


In [ ]:
summary_textrank_cnn = []
summary_pegasus_cnn = []
summary_hybrid_cnn = []
reference_summary_cnn = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_cnn["test"][i]["highlights"]
  reference_summary_cnn.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_cnn["test"][i]["article"]
  result = f_textrank_en(input, 6)

  summary_textrank_cnn.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_cnn,
    predictions = summary_textrank_cnn
)
results

Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vector

{'rouge1': 0.2025194893958502,
 'rouge2': 0.054323702817424165,
 'rougeL': 0.12556750023232488,
 'rougeLsum': 0.15375132442550352}

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_cnn["test"][i]["article"]
  result = f_pegasus_en(input)

  summary_pegasus_cnn.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_cnn,
    predictions = summary_pegasus_cnn
)
results

{'rouge1': 0.32914283406407674,
 'rouge2': 0.1368254624185667,
 'rougeL': 0.23893785031583514,
 'rougeLsum': 0.27385284560246137}

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_cnn["test"][i]["article"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_cnn.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_cnn,
    predictions = summary_hybrid_cnn
)
results

Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vectorization done on cpu
Initializing Bert distilbert-base-uncased
Vector

{'rouge1': 0.2782200016612637,
 'rouge2': 0.08914115383375132,
 'rougeL': 0.19588035089680772,
 'rougeLsum': 0.250945050626913}

PENGUJIAN UNTUK 50 DATA BILLSUM

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_billsum = load_dataset("billsum")

print(f"Features in billsum : {dataset_billsum['train'].column_names}")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Features in billsum : ['text', 'summary', 'title']


In [ ]:
summary_textrank_billsum = []
summary_pegasus_billsum = []
summary_hybrid_billsum = []
reference_summary_billsum = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_billsum["test"][i]["summary"]
  reference_summary_billsum.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_textrank_en(input, 5)

  summary_textrank_billsum.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_billsum,
    predictions = summary_textrank_billsum
)
results

{'rouge1': 0.3610441814236165,
 'rouge2': 0.1732830838057117,
 'rougeL': 0.2178016568277178,
 'rougeLsum': 0.3119172384989168}

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_pegasus_en(input)

  summary_pegasus_billsum.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_billsum,
    predictions = summary_pegasus_billsum
)
results

{'rouge1': 0.31382434619571487,
 'rouge2': 0.14935621091125542,
 'rougeL': 0.223862456525171,
 'rougeLsum': 0.2519800562830645}

In [ ]:
import evaluate

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_billsum["test"][i]["text"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_billsum.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_billsum,
    predictions = summary_hybrid_billsum
)
results

{'rouge1': 0.3215108528972256,
 'rouge2': 0.16248069230927745,
 'rougeL': 0.23034778505163017,
 'rougeLsum': 0.27685553691263454}

PENGUJIAN UNTUK 50 DATA PUBMED

In [ ]:
import evaluate

In [ ]:
# import and load dataset
from datasets import load_dataset

dataset_multi = load_dataset("multi_news")

print(f"Features in multi : {dataset_multi['train'].column_names}")

ModuleNotFoundError: ignored

In [ ]:
summary_textrank_multi = []
summary_pegasus_multi = []
summary_hybrid_multi = []
reference_summary_multi = []

limit = 50

In [ ]:
for i in range (limit):
  # Inisial reference
  reference = dataset_multi["test"][i]["summary"]
  reference_summary_multi.append(reference)

In [ ]:
# textrank rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_textrank_en(input, 4)

  summary_textrank_multi.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_multi,
    predictions = summary_textrank_multi
)
results

{'rouge1': 0.34773997011459734,
 'rouge2': 0.10857454185186804,
 'rougeL': 0.1584172054840794,
 'rougeLsum': 0.18059581927296747}

In [ ]:
# pegasus rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_pegasus_en(input)

  summary_pegasus_multi.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_multi,
    predictions = summary_pegasus_multi
)
results

{'rouge1': 0.2533051705040261,
 'rouge2': 0.08637126935427111,
 'rougeL': 0.15539205380131166,
 'rougeLsum': 0.15408613139596122}

In [ ]:
# hybrid rouge score
for i in range (limit):
  # Inisial kalimat inputan
  input = dataset_multi["test"][i]["document"]
  result = f_hybrid_en(input, 2)

  summary_hybrid_multi.append(result)

rouge = evaluate.load('rouge')
results = rouge.compute(
    references = reference_summary_multi,
    predictions = summary_hybrid_multi
)
results

{'rouge1': 0.2267429758457053,
 'rouge2': 0.06632363722487578,
 'rougeL': 0.13034926689765836,
 'rougeLsum': 0.15795842385062184}